In [1]:
# Import python packages
import pandas as pd
import pyodbc
import requests
import time
import datetime
import sys

In [2]:
def get_sym_data(ticker_str):
    
    API_URL = "https://www.alphavantage.co/query"

    #compact - first 100 lines
    #full - returns all 20 years of data

    t_data = {
        "function": "OVERVIEW",
        "symbol": ticker_str,
        "outputsize": "full",
        "datatype": "json",
        "apikey": "L4W04J8F48N8J7OU"
        }

    response = requests.get(API_URL, t_data)
    response_json = response.json() # maybe redundant

    data = pd.DataFrame.from_dict(response_json, orient= 'index').sort_index(axis=1) 
    
    return data

In [16]:
data = get_sym_data("MSFT")

In [3]:
#x = data[data.index == '52WeekHigh'][0]
#x = data[data.index == '52WeekHigh']['52WeekHigh']
y = data.T
z = y['52WeekHigh'][0]
h = y['52WeekLow'][0]
print([z,h])
# data.to_csv('#WHAT.csv')

NameError: name 'data' is not defined

In [26]:
def get_inc_data(ticker_str):
    
    API_URL = "https://www.alphavantage.co/query"

    #compact - first 100 lines
    #full - returns all 20 years of data

    t_data = {
        "function": "INCOME_STATEMENT",
        "symbol": ticker_str,
        "outputsize": "full",
        "datatype": "json",
        "apikey": "L4W04J8F48N8J7OU"
        }

    response = requests.get(API_URL, t_data)
    response_json = response.json() # maybe redundant

    data = pd.DataFrame.from_dict(response_json['annualReports']).sort_index(axis=1) 

    return data

In [27]:
data = get_inc_data("AAPL")

In [28]:
data.to_csv('temp/income_statement.csv')

In [31]:
def get_bal_data(ticker_str):
    
    API_URL = "https://www.alphavantage.co/query"

    #compact - first 100 lines
    #full - returns all 20 years of data

    t_data = {
        "function": "BALANCE_SHEET",
        "symbol": ticker_str,
        "outputsize": "full",
        "datatype": "json",
        "apikey": "L4W04J8F48N8J7OU"
        }

    response = requests.get(API_URL, t_data)
    response_json = response.json() # maybe redundant

    data = pd.DataFrame.from_dict(response_json['annualReports']).sort_index(axis=1) 

    return data

In [32]:
data = get_bal_data("AAPL")

In [33]:
data.to_csv('temp/Balance_Sheet.csv')

In [35]:
def get_cash_data(ticker_str):
    
    API_URL = "https://www.alphavantage.co/query"

    #compact - first 100 lines
    #full - returns all 20 years of data

    t_data = {
        "function": "CASH_FLOW",
        "symbol": ticker_str,
        "outputsize": "full",
        "datatype": "json",
        "apikey": "L4W04J8F48N8J7OU"
        }

    response = requests.get(API_URL, t_data)
    response_json = response.json() # maybe redundant

    data = pd.DataFrame.from_dict(response_json['annualReports']).sort_index(axis=1) 

    return data

In [36]:
data = get_cash_data("AAPL")

In [37]:
data.to_csv('temp/cash_flow.csv')

In [38]:
overview = pd.read_csv('temp/Overview.csv')
income_statement = pd.read_csv('temp/income_statement.csv')
balance_sheet = pd.read_csv('temp/Balance_Sheet.csv')
cash_flow = pd.read_csv('temp/cash_flow.csv')

In [78]:
#intrinsic value
overview.columns = ['Attribute','Value']
#EPS
EPS = overview[overview['Attribute'] == 'EPS']['Value']
EPS.reset_index(drop=True,inplace=True)
EPS = EPS[0]
#Trailing and Forward PE
TPE = overview[overview['Attribute'] == 'TrailingPE']['Value']
TPE.reset_index(drop=True,inplace=True)
TPE = TPE[0]
FPE = overview[overview['Attribute'] == 'ForwardPE']['Value']
FPE.reset_index(drop=True,inplace=True)
FPE = FPE[0]
#Growth
TPE = float(TPE)
FPE = float(FPE)
Grwth = (TPE - FPE) / TPE 
#10 year look ahead price
EPS = float(EPS)
price2 = EPS * (1 + Grwth) * ((TPE + FPE)/2) 
price1 = EPS * (1 + Grwth) * TPE 
descriptions = ['EPS','Trailing PE','Forward PE','Earnings Growth Rate','Projected PE','Price at Current PE','Price at Proj. PE']
vals = [EPS,TPE,FPE,round(Grwth*100,2),(TPE+FPE)/2,price1,price2]
iv_data = [descriptions,vals]
iv_data = pd.DataFrame(iv_data).T

In [80]:
iv_data.tail()

,0,1
2,Forward PE,28.49
3,Earnings Growth Rate,13.76
4,Projected PE,30.7631
5,Price at Current PE,138.567
6,Price at Proj. PE,129.033


In [101]:
#WACC
EBIT = income_statement.ebit[0]
EBIT = float(EBIT)
interestExpense = income_statement.interestExpense[0]
interestExpense = float(interestExpense) * -1
interest_coverage = EBIT / interestExpense

estimated_spread = {8.5  : 0.68,
                    6.5  : 0.78,
                    5.5  : 0.98,
                    4.25 : 1.08,
                    3    : 1.22,
                    2.5  : 1.56,
                    2.25 : 1.00,
                    2    : 2.40,
                    1.75 : 3.51,
                    1.5  : 4.21,
                    1.25 : 5.15,
                    0.8  : 8.20}

ind = 0

for key, value in estimated_spread.items():
    
    if ind == 0:
        if interest_coverage >= key:
            spread = value
            ind = 1

rf = 0.15
kd = rf + spread
print(kd)

0.8300000000000001


In [3]:
navBar = open('templates\\navBar.html', 'r')
navBar = navBar.read()
print(navBar)

<!DOCTYPE html>
<html>
<head>
<title>Stock Screener</title>
</head>
<body>

<meta name="viewport" content="width=device-width, initial-scale=1">

<link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">

<div class="w3-bar w3-border w3-green">
  <a href="Home" class="w3-bar-item w3-button w3-padding-16">Home</a>
  <a href="About" class="w3-bar-item w3-button w3-padding-16">About</a>
  <a href="https://www.w3schools.com/w3css/default.asp" class="w3-bar-item w3-button w3-padding-16">W3 Schools CSS</a>
  <a href="mailto:bensamborn@comcast.net" class="w3-bar-item w3-button w3-padding-16">Email</a>
  <form action="/Overview" method="post">
  <input type="text" class="w3-bar-item w3-input" placeholder="Search.." width="250" height="75">
</div>


In [4]:
def getBalance(ticker_str):
    
    API_URL = "https://www.alphavantage.co/query"

    #compact - first 100 lines
    #full - returns all 20 years of data

    t_data = {
        "function": "BALANCE_SHEET",
        "symbol": ticker_str,
        "outputsize": "full",
        "datatype": "json",
        "apikey": "L4W04J8F48N8J7OU"
        }

    response = requests.get(API_URL, t_data)
    response_json = response.json() # maybe redundant

    data = pd.DataFrame.from_dict(response_json['annualReports']).sort_index(axis=1) 

    data = data.transpose()

    return data


In [7]:
data = getBalance("AAPL")
print(type(data))

<class 'pandas.core.frame.DataFrame'>
